In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier

Loading Dataset for the developments <br><br>
Dataset contains Crop recomondation data which can be use to predict the good crops for the specified soils and the environmental parameters

In [ ]:
df = pd.read_csv("dataset\Crop_recommendation.csv")

Describe the headers of the dataset data.

In [ ]:
df.head()

Creating the dataframe shape 

In [ ]:
print("Dataframe Shape: ",df.shape)
df.isna().sum()

Check the unique value of the columns

In [ ]:
df['label'].value_counts()

Print a concise summary of a DataFrame.

In [ ]:
df.info()

Describe about data distribution


In [ ]:
df.describe()

Define the types of data

In [ ]:
df.dtypes

Check for null values and NA values in the dataset

In [ ]:
df.isnull().any()

List Number of crops contain in the dataset

In [ ]:
print("Number of crops: ", len(df['label'].unique()))
print("List of crops: ", df['label'].unique())

Summary with pivot table

In [ ]:
crop_summary = pd.pivot_table(cropdf,index=['label'],aggfunc='mean')
crop_summary.head()

Data Distribution

Correlation between different features

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 9))
sns.heatmap(df.corr(), annot=True,cmap='viridis')
ax.set(xlabel='features')
ax.set(ylabel='features')

plt.title('Correlation between different features', fontsize = 15, c='black')
plt.show()

Nitrogen Data Distribution

In [ ]:
sns.displot(x=df['N'], bins=20,kde=True,edgecolor="black",color='black',facecolor='#ea5545')
plt.title("Nitrogen",size=18)
plt.show()

Phosphorus Data Distribution

In [ ]:
sns.displot(x=df['P'],bins=20,color='black',edgecolor='black',kde=True,facecolor='#ea5545')
plt.title("Phosphorus", size=18)
plt.xticks(range(0,150,20))
plt.show()

Potassium Data Distribution

In [ ]:
sns.displot(x=df['K'],kde=True, bins=20, facecolor='#ea5545',edgecolor='black', color='black')
plt.title("Potassium",size=18)
plt.show()

Temperature Data Distribution

In [ ]:
sns.displot(x=df['temperature'], bins=20,kde=True,edgecolor="black",color='black',facecolor='#ea5545')
plt.title("Temperature",size=18)
plt.show()

Humidity Data Distribution

In [ ]:
sns.displot(x=df['humidity'], color='black',facecolor='#ea5545',kde=True,edgecolor='black')
plt.title("Humidity",size=18)
plt.show()

Rainfall Data Distribution

In [ ]:
sns.displot(x=df['rainfall'], color='black',facecolor='#ea5545',kde=True,edgecolor='black')
plt.title("Rainfall",size=18)
plt.show()

 Categorical plot 

In [ ]:
sns.relplot(x='rainfall',y='temperature',data=df,kind='scatter',hue='label',height=5)
plt.title("Temperature", size=18)
plt.show()

In [ ]:
sns.pairplot(data=df,hue='label')
plt.show()

Outerlier detection using graphs



In [ ]:
# Unique values in the label column

crops = df['label'].unique()
print(len(crops))
print(crops)
print(pd.value_counts(df['label']))

In [ ]:
# Filtering each unique label and store it in a list df2 for to plot the box plot

df2=[]
for i in crops:
    df2.append(df[df['label'] == i])
df2[1].head()

In [ ]:
sns.catplot(data=df, x='label', y='temperature', kind='box', height=10, aspect=20/8.27)
plt.title("Temperature", size=18)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='humidity', kind='box', height=10, aspect=20/8.27)
plt.title("Humidity", size=18)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='temperature', kind='box', height=10, aspect=20/8.27)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='N', kind='box', height=10, aspect=20/8.27)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='ph', kind='box', height=10, aspect=20/8.27)
plt.title("Nitrogen",size=18)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='P', kind='box', height=10, aspect=20/8.27)
plt.title("Phosphorus",size=18)
plt.show()

In [ ]:
sns.catplot(data=df, x='label', y='K', kind='box', height=10, aspect=20/8.27)
plt.title("Potassium",size=20)
plt.show()

These graphs shows that thers is no outliers present in this dataset



Lets check through Mathematics (Statistics)

In [ ]:
def detect_outlier(x):
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    IQR = q3-q1
    min_limit = q1 - (1.5*IQR)
    max_limit = q3 + (1.5*IQR)
    print(f"Minimum limit: {min_limit} Maximum limit: {max_limit}")
    print(f"Minimum value: {x.min()}   MAximum Value: {x.max()}")
    for i in [x.min(),x.max()]:
        if i == x.min():
            if min_limit > x.min():
                print("Minimum limit failed - Need to remove minimum value")
            elif min_limit < x.min():
                print("Minimum limit passed - No need to remove outlier")
        elif i == x.max():
            if max_limit > x.max():
                print("Maximum limit passed - No need to remove outlier")
            elif max_limit < x.max():
                print("Maximum limit failed - Need to remove maximum value")
detect_outlier(df['K'][df['label']=='grapes'])

In [ ]:
for i in df['label'].unique():
    detect_outlier(df['K'][df['label']==i])
    print('---------------------------------------------')

Prediction - Splitting of Test and traning data

In [ ]:
x = df.drop(['label'], axis=1)
x.head()

In [ ]:
Y = df['label']
encode = preprocessing.LabelEncoder()
y = encode.fit_transform(Y)
print("Label length: ",len(y))

In [ ]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y)
print(len(x_train),len(y_train),len(x_test),len(y_test))

Choosing the best model

In [ ]:
a={'decision tree' : {
        'model' : DecisionTreeClassifier(criterion='gini'),
        'params':{'decisiontreeclassifier__splitter':['best','random']}
    },
    'svm': {
        'model': SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
   'k classifier':{
       'model':KNeighborsClassifier(),
       'params':{'kneighborsclassifier__n_neighbors':[5,10,20,25],'kneighborsclassifier__weights':['uniform','distance']}
   }
}

Decision Tree, Support vector mechanism, Random forest is being used here